In [1]:
import pandas as pd
import numpy as np

from average_conn_V3 import average_conn

# Load data

In [2]:
from nilearn.datasets import fetch_icbm152_brain_gm_mask, fetch_development_fmri, fetch_atlas_basc_multiscale_2015
import nibabel as nib

# Load mask of parcels
parcellation = fetch_atlas_basc_multiscale_2015(version='sym')['scale064']
img_mist = nib.load(parcellation)

# Grey matter segmentation from the MNI template 
mask_mni = fetch_icbm152_brain_gm_mask()

# Fetch demo dataset
data = fetch_development_fmri(n_subjects=1, reduce_confounds=True)
imgs = data.func[0:10]
confounds = data.confounds[0:10]

# test private functions

In [3]:
avg_conn = average_conn(imgs, mask_mni, img_mist, confounds)

## It works on simple data

# Now for whole-brain intra-network average connectivity

Load fMRI data for one subject

Compute the time series using mist resampled

Compute the AFC and also get the parcel sizes